<h1>Perform Federated Learning at Grid Platform </h1>
In this notebook, we will train a model using federated approach.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

**NOTE:**
Components:
 - Grid Gateway(http://localhost:8080)
 - Grid Node Bob (http://localhost:3000)
 - Grid Node Alice (http://localhost:3001)
 - Grid Node Bill (http://localhost:3002)

This notebook was made based on <a href="https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2010%20-%20Federated%20Learning%20with%20Secure%20Aggregation.ipynb">Part 10: Federated Learning with Encrypted Gradient Aggregation</a> tutorial

<h2>Import dependencies</h2>

In [1]:
import grid as gr
import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

/home/marcus/Desktop/PyGrid/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/marcus/Desktop/PyGrid/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/marcus/Desktop/PyGrid/env37/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


<h2>Define Model</h2>

In [2]:
hook = sy.TorchHook(th)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

<h2> Connect to Grid Network</h2>

In [3]:
GRID_ADDRESS = 'localhost'
GRID_PORT = '5000'

my_grid = gr.GridNetwork("http://" + GRID_ADDRESS + ":" + GRID_PORT)

<h2>Search a dataset</h2>
Here, we will use the gateway to look for which grid nodes have the data with the following tags.

In [4]:
data = my_grid.search("#X", "#mnist", "#dataset")
target = my_grid.search("#Y", "#mnist", "#dataset")

In [5]:
data

[[(Wrapper)>[PointerTensor | me:16183473676 -> Alice:75937734335],
  (Wrapper)>[PointerTensor | me:8972585377 -> Alice:13369193585],
  (Wrapper)>[PointerTensor | me:72984223000 -> Alice:73360563251],
  (Wrapper)>[PointerTensor | me:14784884248 -> Alice:73397553804],
  (Wrapper)>[PointerTensor | me:84279669773 -> Alice:56299750911],
  (Wrapper)>[PointerTensor | me:66978650288 -> Alice:48969409093],
  (Wrapper)>[PointerTensor | me:61267140964 -> Alice:35265843619],
  (Wrapper)>[PointerTensor | me:12054518694 -> Alice:41639212052],
  (Wrapper)>[PointerTensor | me:79330006315 -> Alice:79375379845],
  (Wrapper)>[PointerTensor | me:86947260186 -> Alice:68680154976],
  (Wrapper)>[PointerTensor | me:65152420750 -> Alice:66200966358],
  (Wrapper)>[PointerTensor | me:16054691290 -> Alice:44708340259],
  (Wrapper)>[PointerTensor | me:48231522862 -> Alice:34948846404],
  (Wrapper)>[PointerTensor | me:81679720365 -> Alice:89811736933],
  (Wrapper)>[PointerTensor | me:94188979418 -> Alice:6231335702

<h2>Perform Train</h2>
The code below will train the previously defined model with the data belonging to each grid node.

In [6]:
def epoch_total_size(data):
    total = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            total += data[i][j].shape[0]
            
    return total

In [7]:
N_EPOCS = 3
SAVE_MODEL = True
SAVE_MODEL_PATH = './models'

def train(epoch):
    model.train()
    epoch_total = epoch_total_size(data)
    current_epoch_size = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            current_epoch_size += len(data[i][j])
            worker = data[i][j].location
            model.send(worker)
            optimizer.zero_grad()
            pred = model(data[i][j])
            loss = criterion(pred, target[i][j])
            loss.backward()
            optimizer.step()
            model.get()
            loss = loss.get()
            print('Train Epoch: {} | With {} data |: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                      epoch, worker.id, current_epoch_size, epoch_total,
                            100. *  current_epoch_size / epoch_total, loss.item()))
                    
for epoch in range(N_EPOCS):
    train(epoch)

if(SAVE_MODEL):
    print("Saving model")
    th.save(model.state_dict(), SAVE_MODEL_PATH + "/fedmodel.pt")


Train Epoch: 0 | With Alice data |: [1000/60000 (2%)]	Loss: 2.323812
Train Epoch: 0 | With Alice data |: [2000/60000 (3%)]	Loss: 2.313196
Train Epoch: 0 | With Alice data |: [3000/60000 (5%)]	Loss: 2.312180
Train Epoch: 0 | With Alice data |: [4000/60000 (7%)]	Loss: 2.302857
Train Epoch: 0 | With Alice data |: [5000/60000 (8%)]	Loss: 2.299912
Train Epoch: 0 | With Alice data |: [6000/60000 (10%)]	Loss: 2.299375
Train Epoch: 0 | With Alice data |: [7000/60000 (12%)]	Loss: 2.290473
Train Epoch: 0 | With Alice data |: [8000/60000 (13%)]	Loss: 2.289073
Train Epoch: 0 | With Alice data |: [9000/60000 (15%)]	Loss: 2.285751
Train Epoch: 0 | With Alice data |: [10000/60000 (17%)]	Loss: 2.272835
Train Epoch: 0 | With Alice data |: [11000/60000 (18%)]	Loss: 2.267965
Train Epoch: 0 | With Alice data |: [12000/60000 (20%)]	Loss: 2.268417
Train Epoch: 0 | With Alice data |: [13000/60000 (22%)]	Loss: 2.259840
Train Epoch: 0 | With Alice data |: [14000/60000 (23%)]	Loss: 2.252531
Train Epoch: 0 | Wit

Websocket connection closed (worker: Bob)


ConnectionResetError: [Errno 104] Connection reset by peer

<h2>Disconnect Grid Nodes</h2>

In [ ]:
my_grid.disconnect_nodes()